## Интеллектуальные системы и технологии  
Практика №7. RNN, LSTM, GRU  
Жаров Всеволод, ИММО-01-23

In [2]:
import wget
import pandas as pd
import numpy as np
from sklearn import preprocessing
import torch
from torch import nn

C:\Users\wsewo\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Гиперпараметры

In [3]:
train_name = "http://azuremlsamples.azureml.net/templatedata/PM_train.txt"
test_name = "http://azuremlsamples.azureml.net/templatedata/PM_test.txt"
truth_name = "http://azuremlsamples.azureml.net/templatedata/PM_truth.txt"

raw_train_dest = "./data/azure/raw_train.txt"
raw_test_dest = "./data/azure/raw_test.txt"
raw_truth_dest = "./data/azure/raw_truth.txt"

train_dest = "./data/azure/train.csv"
test_dest = "./data/azure/test.csv"

time_lenght = 50
id_var = 5

### Предобработка

In [41]:
# wget.download(url=train_name, out=raw_train_dest)
# wget.download(url=test_name, out=raw_test_dest)
# wget.download(url=truth_name, out=raw_truth_dest)

'./data/azure/test.txt'

In [125]:
column_names = [
    'id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
    's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
    's15', 's16', 's17', 's18', 's19', 's20', 's21'
]

In [126]:
# read training data 
train_df = pd.read_csv(raw_train_dest, sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = column_names

# read test data
test_df = pd.read_csv(raw_test_dest, sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = column_names


# read ground truth data
truth_df = pd.read_csv(raw_truth_dest, sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [127]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

id  cycle  setting1  setting2  setting3      s1      s2       s3       s4  \
0   1      1   -0.0007   -0.0004     100.0  518.67  641.82  1589.70  1400.60   
1   1      2    0.0019   -0.0003     100.0  518.67  642.15  1591.82  1403.14   
2   1      3   -0.0043    0.0003     100.0  518.67  642.35  1587.99  1404.20   
3   1      4    0.0007    0.0000     100.0  518.67  642.35  1582.79  1401.87   
4   1      5   -0.0019   -0.0002     100.0  518.67  642.37  1582.85  1406.22   

      s5  ...     s12      s13      s14     s15   s16  s17   s18    s19  \
0  14.62  ...  521.66  2388.02  8138.62  8.4195  0.03  392  2388  100.0   
1  14.62  ...  522.28  2388.07  8131.49  8.4318  0.03  392  2388  100.0   
2  14.62  ...  522.42  2388.03  8133.23  8.4178  0.03  390  2388  100.0   
3  14.62  ...  522.86  2388.08  8133.83  8.3682  0.03  392  2388  100.0   
4  14.62  ...  522.19  2388.04  8133.80  8.4294  0.03  393  2388  100.0   

     s20      s21  
0  39.06  23.4190  
1  39.00  23.4236  
2  38.95  23.3442  
3  38.88  23.3739  
4  38.90  23.4044  

[5 rows x 26 columns]

In [128]:
print(len(train_df))
print(np.max(train_df["cycle"]))
print(np.max(train_df["id"]))

20631
362
100


In [129]:
test_df = test_df.sort_values(['id','cycle'])
test_df.head()

id  cycle  setting1  setting2  setting3      s1      s2       s3       s4  \
0   1      1    0.0023    0.0003     100.0  518.67  643.02  1585.29  1398.21   
1   1      2   -0.0027   -0.0003     100.0  518.67  641.71  1588.45  1395.42   
2   1      3    0.0003    0.0001     100.0  518.67  642.46  1586.94  1401.34   
3   1      4    0.0042    0.0000     100.0  518.67  642.44  1584.12  1406.42   
4   1      5    0.0014    0.0000     100.0  518.67  642.51  1587.19  1401.92   

      s5  ...     s12      s13      s14     s15   s16  s17   s18    s19  \
0  14.62  ...  521.72  2388.03  8125.55  8.4052  0.03  392  2388  100.0   
1  14.62  ...  522.16  2388.06  8139.62  8.3803  0.03  393  2388  100.0   
2  14.62  ...  521.97  2388.03  8130.10  8.4441  0.03  393  2388  100.0   
3  14.62  ...  521.38  2388.05  8132.90  8.3917  0.03  391  2388  100.0   
4  14.62  ...  522.15  2388.03  8129.54  8.4031  0.03  390  2388  100.0   

     s20      s21  
0  38.86  23.3735  
1  39.02  23.3916  
2  39.08  23.4166  
3  39.00  23.3737  
4  38.99  23.4130  

[5 rows x 26 columns]

In [130]:
print(len(test_df))
print(np.max(test_df["cycle"]))
print(np.max(test_df["id"]))

13096
303
100


In [131]:
truth_df.head()

0
0  112
1   98
2   69
3   82
4   91

In [132]:
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

id  cycle  setting1  setting2  setting3      s1      s2       s3       s4  \
0   1      1   -0.0007   -0.0004     100.0  518.67  641.82  1589.70  1400.60   
1   1      2    0.0019   -0.0003     100.0  518.67  642.15  1591.82  1403.14   
2   1      3   -0.0043    0.0003     100.0  518.67  642.35  1587.99  1404.20   
3   1      4    0.0007    0.0000     100.0  518.67  642.35  1582.79  1401.87   
4   1      5   -0.0019   -0.0002     100.0  518.67  642.37  1582.85  1406.22   

      s5  ...      s13      s14     s15   s16  s17   s18    s19    s20  \
0  14.62  ...  2388.02  8138.62  8.4195  0.03  392  2388  100.0  39.06   
1  14.62  ...  2388.07  8131.49  8.4318  0.03  392  2388  100.0  39.00   
2  14.62  ...  2388.03  8133.23  8.4178  0.03  390  2388  100.0  38.95   
3  14.62  ...  2388.08  8133.83  8.3682  0.03  392  2388  100.0  38.88   
4  14.62  ...  2388.04  8133.80  8.4294  0.03  393  2388  100.0  38.90   

       s21  RUL  
0  23.4190  191  
1  23.4236  190  
2  23.3442  189  
3  23.3739  188  
4  23.4044  187  

[5 rows x 27 columns]

In [133]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)

# generate label columns for training data
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2

# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

id  cycle  setting1  setting2  setting3   s1        s2        s3        s4  \
0   1      1  0.459770  0.166667       0.0  0.0  0.183735  0.406802  0.309757   
1   1      2  0.609195  0.250000       0.0  0.0  0.283133  0.453019  0.352633   
2   1      3  0.252874  0.750000       0.0  0.0  0.343373  0.369523  0.370527   
3   1      4  0.540230  0.500000       0.0  0.0  0.343373  0.256159  0.331195   
4   1      5  0.390805  0.333333       0.0  0.0  0.349398  0.257467  0.404625   

    s5  ...  s16       s17  s18  s19       s20       s21  RUL  label1  label2  \
0  0.0  ...  0.0  0.333333  0.0  0.0  0.713178  0.724662  191       0       0   
1  0.0  ...  0.0  0.333333  0.0  0.0  0.666667  0.731014  190       0       0   
2  0.0  ...  0.0  0.166667  0.0  0.0  0.627907  0.621375  189       0       0   
3  0.0  ...  0.0  0.333333  0.0  0.0  0.573643  0.662386  188       0       0   
4  0.0  ...  0.0  0.416667  0.0  0.0  0.589147  0.704502  187       0       0   

   cycle_norm  
0     0.00000  
1     0.00277  
2     0.00554  
3     0.00831  
4     0.01108  

[5 rows x 30 columns]

In [134]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)

# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)

# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

id  cycle  setting1  setting2  setting3   s1        s2        s3        s4  \
0   1      1  0.632184  0.750000       0.0  0.0  0.545181  0.310661  0.269413   
1   1      2  0.344828  0.250000       0.0  0.0  0.150602  0.379551  0.222316   
2   1      3  0.517241  0.583333       0.0  0.0  0.376506  0.346632  0.322248   
3   1      4  0.741379  0.500000       0.0  0.0  0.370482  0.285154  0.408001   
4   1      5  0.580460  0.500000       0.0  0.0  0.391566  0.352082  0.332039   

    s5  ...  s16       s17  s18  s19       s20       s21  cycle_norm  RUL  \
0  0.0  ...  0.0  0.333333  0.0  0.0  0.558140  0.661834     0.00000  142   
1  0.0  ...  0.0  0.416667  0.0  0.0  0.682171  0.686827     0.00277  141   
2  0.0  ...  0.0  0.416667  0.0  0.0  0.728682  0.721348     0.00554  140   
3  0.0  ...  0.0  0.250000  0.0  0.0  0.666667  0.662110     0.00831  139   
4  0.0  ...  0.0  0.166667  0.0  0.0  0.658915  0.716377     0.01108  138   

   label1  label2  
0       0       0  
1       0       0  
2       0       0  
3       0       0  
4       0       0  

[5 rows x 30 columns]

### Сохранение датасета

In [137]:
train_df.to_csv(train_dest)
test_df.to_csv(test_dest)

### Загрузка датасета

In [4]:
train_df = pd.read_csv(train_dest)
test_df = pd.read_csv(test_dest)

### Создание нужного представления данных для модели

In [5]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
    
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

def get_seq_and_labels(df, id_var, sequence_length, sequence_cols):
    # seq_gen = (list(gen_sequence(df[df['id']==id], sequence_length, sequence_cols)) for id in df['id'].unique())
    # seq_gen = np.concatenate(list(seq_gen)).astype(float)
    seq_gen = (list(gen_sequence(df[df['id']==id_var], sequence_length, sequence_cols)))
    seq_gen = torch.tensor(seq_gen)

    # label_gen = [gen_labels(df[df['id']==id], sequence_length, ['label1']) for id in df['id'].unique()]
    label_gen = [gen_labels(df[df['id']==id_var], sequence_length, ['label1'])]
    label_array = np.concatenate(label_gen, dtype=float)
    new_label = [float(x) for x in label_array]
    new_label = torch.tensor(new_label).unsqueeze(1)

    return seq_gen, new_label

In [6]:
def full_get_seq_and_labels(df, id_var, time_lenght, sequence_cols):
    if id_var is not None:
        return get_seq_and_labels(df, id_var, time_lenght, sequence_cols)
    
    id_var = np.linspace(2, 100)
    seq, lab = get_seq_and_labels(df, 1, time_lenght, sequence_cols)
    for id in id_var:
        seq_train, labels_train = get_seq_and_labels(df, id, time_lenght, sequence_cols)
        seq = torch.cat((seq, seq_train), dim=0)
        lab = torch.cat((lab, labels_train), dim=0)
    return seq, lab

In [7]:
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

seq_train, labels_train = full_get_seq_and_labels(train_df, id_var, time_lenght, sequence_cols)
seq_test, labels_test = full_get_seq_and_labels(test_df, id_var, time_lenght, sequence_cols)
print("Тренировочные данные:")
print(seq_train.shape)
print(labels_train.shape)
print("Тестовые данные:")
print(seq_test.shape)
print(labels_test.shape)

print("------")

seq_train, labels_train = full_get_seq_and_labels(train_df, None, time_lenght, sequence_cols)
seq_test, labels_test = full_get_seq_and_labels(test_df, None, time_lenght, sequence_cols)
print("Тренировочные данные:")
print(seq_train.shape)
print(labels_train.shape)
print("Тестовые данные:")
print(seq_test.shape)
print(labels_test.shape)

C:\Users\wsewo\AppData\Local\Temp\ipykernel_11344\4015712693.py:21: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  seq_gen = torch.tensor(seq_gen)


Тренировочные данные:
torch.Size([219, 50, 25])
torch.Size([219, 1])
Тестовые данные:
torch.Size([48, 50, 25])
torch.Size([48, 1])
------
Тренировочные данные:
torch.Size([8059, 50, 25])
torch.Size([8059, 1])
Тестовые данные:
torch.Size([4359, 50, 25])
torch.Size([4359, 1])


### Проверка выходов модели

In [181]:
input_size = 25
hidden_size = 100
num_layers = 2
drop = 0.2

In [209]:
rnn = nn.LSTM(
        input_size = input_size, 
        hidden_size = hidden_size, 
        num_layers = num_layers,
        batch_first = True,
        dropout = drop,
        bidirectional = False
    )

lin = nn.Linear(
    in_features = time_lenght,
    out_features=1
)

In [210]:
print(seq_train.shape)

y_hat, _ = rnn(seq_train)
print(y_hat.shape)

y_hat = y_hat[:, :, -1]
print(y_hat.shape)

y_hat = lin(y_hat)
print(y_hat.shape)

torch.Size([219, 50, 25])
torch.Size([219, 50, 100])
torch.Size([219, 50])
torch.Size([219, 1])


### Обучение

Для обучения надо запустить файл train.py из папки nn_module. При желании можно отредактировать конфиг.  
Сам процесс обучения логируется на [wandb](https://wandb.ai/lost_in_thoughts/rnn/workspace?workspace=user-wsewolod)